In [1]:
import pandas as pd
import numpy as np

features_all = pd.read_csv('features_all_weeks.csv')
features_all.drop(columns = ['Unnamed: 0', 'X1', 'X'], inplace = True)
# display(features_all)
print(features_all.info())

unclean_train_df = features_all[features_all['week'] != 8]
unclean_test_df = features_all[features_all['week'] == 8]

unclean_train_df.drop(columns = ['week'], inplace = True)
unclean_test_df.drop(columns = ['week'], inplace = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274529 entries, 0 to 274528
Data columns (total 28 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   playId                 274529 non-null  int64  
 1   gameId                 274529 non-null  int64  
 2   frameId                274529 non-null  int64  
 3   area_block             274529 non-null  float64
 4   dist_block             274529 non-null  float64
 5   dist_to_qb_block       274529 non-null  float64
 6   x_force_block          274529 non-null  float64
 7   y_force_block          274529 non-null  float64
 8   norm_weighted_x_block  274529 non-null  float64
 9   norm_weighted_y_block  274529 non-null  float64
 10  area_rush              274529 non-null  float64
 11  dist_rush              274529 non-null  float64
 12  dist_to_qb_rush        274529 non-null  float64
 13  x_force_rush           274529 non-null  float64
 14  y_force_rush           274529 non-nu

C:\Users\Harrison\AppData\Local\Temp\ipykernel_18232\2618881155.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unclean_train_df.drop(columns = ['week'], inplace = True)
C:\Users\Harrison\AppData\Local\Temp\ipykernel_18232\2618881155.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unclean_test_df.drop(columns = ['week'], inplace = True)


In [ ]:
# keep track of frameID
def df_split_by_frame(df, group_by = 5, features = 'all'):
    df_grouped = df.groupby('frameId')
    df_dict = dict()

    for n, df in df_grouped:
        if (n % group_by == 1 and n < 60):
            y = df['bad_thing']
            x = df.drop(columns = ['bad_thing', 'hit', 'hurry', 'sack', 'playId', 'gameId', 'frameId'])
            # display(x)
            df_dict[n] = (x, y)
            # df_list.append((n, x, y))
    return df_dict

In [ ]:
from sklearn.ensemble import RandomForestClassifier

def train_rf_model(df_dict):
    models = dict()
    for (frameID, (x, y)) in df_dict.items():
        print(frameID)
        rf = RandomForestClassifier(n_estimators=500, oob_score=True, random_state=123456, max_depth = 15)
        rf.fit(x, y)
        models[frameID] = rf
        del rf
    return models

def train_lr_model(df_list):
    models = []
    for (frameID, x, y) in df_list:
        # TODO: change to LR model with intercept
        rf = RandomForestClassifier(n_estimators=500, oob_score=True, random_state=123456, max_depth = 15)
        rf.fit(x, y)
        models.append((frameID, rf))
        del rf
    return models

from sklearn.metrics import accuracy_score, recall_score
def evaluate(test_df_dict, models):
    for frameID, model in models.items():
        x, y = test_df_dict[frameID]
        pred = model.predict(x)
        accuracy = accuracy_score(y, pred)
        recall = recall_score(y, pred)
        
        print(f'{frameID}: Out-of-bag score estimate: {model.oob_score_:.3}')
        print(f'{frameID}: Mean accuracy score: {accuracy:.3}')
        print(f'{frameID}: Recall: {recall:.3}')


In [ ]:
clean_train_df_dict = df_split_by_frame(unclean_train_df)
clean_test_df_dict = df_split_by_frame(unclean_test_df)

models = train_rf_model(clean_train_df_dict)
pickle.dump(models, open("model_list_all_features.pkl", "wb"))

In [ ]:
models = pickle.load(open("model_list_all_features.pkl", "rb"))

In [ ]:
evaluate(clean_test_df_dict, models)